In [1]:
import numpy as np
np.set_printoptions(legacy='1.25')
import tensorflow as tf

c:\Program Files\Python313\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Preprocessing, encoding etc.

In [3]:
# After doing the data processing, one hot encoding etc , build a CNN model and measure its accuracy and try to optimize the model

x_train_norm = (x_train).reshape((60000,28,28,1)) / 255 # reshaping and normalizing the data
x_test_norm = (x_test).reshape((10000,28,28,1)) / 255

y_train_enc = tf.keras.utils.to_categorical(y_train) # one hot encoding the targets
y_test_enc = tf.keras.utils.to_categorical(y_test)

### Making the model

In [20]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1), padding='same'))
# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(tf.keras.layers.Dropout(0.3))
# model.add(tf.keras.layers.Conv2D(8, (3,3), activation='relu', padding='same'))
# model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │       250,890 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 251,210 (981.29 KB)

 Trainable params: 251,210 (981.29 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(x_train_norm, y_train_enc, validation_split=0.15, batch_size=10, epochs=10)

Epoch 1/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9513 - loss: 0.1655 - val_accuracy: 0.9772 - val_loss: 0.0804
Epoch 2/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9769 - loss: 0.0740 - val_accuracy: 0.9821 - val_loss: 0.0714
Epoch 3/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9831 - loss: 0.0543 - val_accuracy: 0.9817 - val_loss: 0.0693
Epoch 4/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9872 - loss: 0.0396 - val_accuracy: 0.9829 - val_loss: 0.0694
Epoch 5/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9897 - loss: 0.0318 - val_accuracy: 0.9821 - val_loss: 0.0700
Epoch 6/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9918 - loss: 0.0250 - val_accuracy: 0.9822 - val_loss: 0.0816
Epoch 7/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9932 - loss: 0.0206 - val_accuracy: 0.9810 - val_loss: 0.0835
Epoch 8/10
5100/5100 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.9939 - loss: 0

In [23]:
pred_data = model.predict(x_test_norm)
correct_pred_index = np.argmax(pred_data, axis=1)
correct_test_index = np.argmax(y_test_enc, axis=1)
accuracy = np.mean(correct_pred_index == correct_test_index) * 100
accuracy

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 860us/step


97.99

## Performance:

f: Flatten layer <br/>
p: Max Polling layer <br />
OF: Overfit
| Hidden Layers  | Dropouts    | Epochs  | Accuracy    | Time   |
| -------------- | ----------- | ------- | ----------- | ------ |
| 2 (32,f)       | None        | 10      | 97.92%      | 1m 49s |
| 4 (16,p,8,f)   | None        | 10      | 98.71%      | 1m 25s |
| 4 (16,p,8,f)   | None        | 20      | 98.66%      | 2m 47s |
| 4 (16,p,8,f)   | 2 (0.2,0.2) | 10      | 98.75%      | 1m 31s |
| 4 (16,p,8,f)   | 2 (0.2,0.2) | 20      | 98.79%      | 3m 03s |
| 4 (16,p,8,f)   | 2 (0.4,0.2) | 20      | 98.78%      | 3m 03s |
| 2 (16,f)       | 1 (0.3)     | 20      | 98.32%      | 2m 42s |
| 2 (32,f)       | 1 (0.3)     | 10      | 97.99%      | 2m 42s |

Basically dropout layers are not having any effect in this particular model.